In [2]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
# Author: spopov@google.com (Stefan Popov)

WORKDIR = "/path/to/cad_estate/"

import asyncio
import io
import json
import os
import sys
sys.path.append(os.path.join(WORKDIR, 'src'))

import ipywidgets
import nest_asyncio
import numpy as np
from IPython import display

from cad_estate import debug_helpers
from cad_estate import file_system as fs
from cad_estate import frames as frame_lib
from cad_estate import room_structure as struct_lib
from cad_estate.notebooks import room_structure_notebook_helper as helper_lib

ANNOTATIONS_DIR = os.path.join(WORKDIR, "data/annotations")

# If set to None, frames will be downloaded on the fly
# FRAMES_DIR = None
FRAMES_DIR = os.path.join(WORKDIR, "data/frames")

# The number of frames to display
NUM_FRAMES = 10

# The width of the displayed images
IMAGE_WIDTH = 384

# Several room structure examples
EXAMPLE_SCENES = [
    # Clips from the video accompanying the paper
    "1SaNI8rXoOo_68702000", "0I8Fpn1pm9w_35668967", "3tpI2SLW-vw_220920000",
    "00ccbtp2aSQ_130197000", "4hALIRqlDQ0_64698000", "6GgUTiECyZE_140607000",
    "6Coxyzoemdg_74307000",

    # Further clips
    "-elAMee-28E_45011633", "0Z_z6B4vSNY_118418418", "1CtWZskj7Is_197664133",
    "0Y9HuQ1fm8U_230830600", "00rMZpGSeOI_128662000", "-_02xnW0mm8_157457000",
    "4VJAjU97J30_126993533", "0mPtI07cVRs_171004000", "0Z_z6B4vSNY_128828829",
    "2Uvmj0b_xi0_153720233", "0l2fu_lBI6o_213913700", "52ny7BGE1mU_30663997",
    "-pi6_f_9y4U_83680000", "15ko-w6mwXw_34833333", "33h8QKVktKE_149550000",
    "3lMpvKbW4x8_65298633", "6lj2EBPG0GM_91391300", "-x4_aTnBpQI_157357000",
]  # yapf: disable


# One time setup
nest_asyncio.apply()
ANNOTATIONS_DIR = fs.abspath(ANNOTATIONS_DIR)
disp = debug_helpers.display_images
# Render the notebook using the full width of the screen
display.display(
    display.HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
AnnotationType = helper_lib.AnnotationType
scene_name, frame_index, annotation_type = (
    helper_lib.create_interactive_widgets(EXAMPLE_SCENES, NUM_FRAMES,
                                          ANNOTATIONS_DIR))
mix_alpha = ipywidgets.FloatSlider(0.5, min=0.0, max=1.0, step=0.1)


@ipywidgets.interact
def render_frame(scene_name=scene_name, frame_index=frame_index,
                 annotations_type=annotation_type, mix_alpha=mix_alpha):
  # Load the frame and the room annotations
  frames_json, room_npz = asyncio.run(
      fs.read_all_bytes_async(
          fs.join(ANNOTATIONS_DIR, scene_name, v)
          for v in ["frames.json", "room_structure.npz"]))
  frames = frame_lib.load_metadata(json.loads(frames_json))
  room = struct_lib.load_room_structure(np.load(io.BytesIO(room_npz)))

  # Filter frames and load their images
  frames_dir = helper_lib.download_frames(frames, FRAMES_DIR)
  if annotations_type == AnnotationType.NONE:
    mask = frame_lib.sample_regular(frames, NUM_FRAMES)
  else:
    mask = struct_lib.annotated_frames_mask(room, frames)
  frames = frame_lib.filter(frames, mask)
  frames = asyncio.run(frame_lib.load_images(frames, frames_dir))

  # Render the room structure geometry
  ii, rgb = helper_lib.render_frame(frames, room, frame_index, IMAGE_WIDTH)
  disp_elements = [
      ii, "Rendered room structure", rgb, "Video frame",
      ii * mix_alpha + rgb * (1 - mix_alpha), "Mixed render+frame"
  ]

  # Load and display the annotations if requested
  if annotations_type != AnnotationType.NONE:
    raw = annotations_type == AnnotationType.RAW
    annotations = asyncio.run(
        struct_lib.load_annotations(room, frames, ANNOTATIONS_DIR, raw))
    str_ann, vis_ann = helper_lib.render_annotations(annotations, frame_index,
                                                     IMAGE_WIDTH)
    disp_elements += [
        str_ann * mix_alpha + rgb * (1 - mix_alpha), "Structure annotation",
        vis_ann * mix_alpha + rgb * (1 - mix_alpha), "Visible annotation"
    ]

  disp(*disp_elements)
  display.display(display.HTML("<br/>"))
  display.display(display.HTML(helper_lib.get_legend_html(room)))